<a href="https://colab.research.google.com/github/ThesanR/TorontoPermitClassification/blob/main/PermitClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
#2015 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/fbceeefc-a24d-43d0-8009-a1ec872550d0/download/Cleared%20Permits%202015.zip
!unzip -q "Cleared Permits 2015.zip"
#2016 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/b6f1907e-5a45-43f7-8c3c-56e4edc4f42f/download/Cleared%20Permits%202016.zip
!unzip -q "Cleared Permits 2016.zip"
#2017-2023 Cleared Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/building-permits-cleared-permits/resource/23de66da-ed1a-434a-a0fa-d85ec55f8f20/download/Cleared%20Building%20Permits%20since%202017.csv
#Active Permits
!wget -q https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/108c2bd1-6945-46f6-af92-02f5658ee7f7/resource/dfce3b7b-4f17-4a9d-9155-5e390a5ffa97/download/building-permits-active-permits.csv

replace clearedpermits2015.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace clearedpermits2016.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


Classification of Permit applications from the City of Toronto between the years 2000 and 2023.

In [114]:
import pandas as pd
import numpy as np

df2015 = pd.read_csv('clearedpermits2015.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df2016 = pd.read_csv('clearedpermits2016.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df2017_2023 = pd.read_csv('Cleared Building Permits since 2017.csv', dtype={"POSTAL": str, "REVISION_NUM": str})
df = pd.concat([df2015, df2016, df2017_2023], ignore_index=True, sort=False)
df = df.drop(df[(df.POSTAL == "   ") | (df.EST_CONST_COST == "DO NOT UPDATE OR DELETE THIS INFO FIELD") | (df.STATUS != "Closed")].index)
df = df[['PERMIT_NUM', 'PERMIT_TYPE', 'STRUCTURE_TYPE', 'WORK', 'POSTAL', 'APPLICATION_DATE', 'ISSUED_DATE', 'EST_CONST_COST']]
df = df.dropna()

#df['EST_CONST_COST'] = ((df['EST_CONST_COST'].replace(",","", regex=True)).astype(float)).astype(int)
dftemp = ((df['EST_CONST_COST'].replace(",","", regex=True)).astype(float)).astype(int)
df = df.drop('EST_CONST_COST', axis=1)
df = pd.concat([df, dftemp], axis=1)
df = df.drop(df[df.EST_CONST_COST < 100].index)

df = df[~df['PERMIT_TYPE'].isin((df['PERMIT_TYPE'].value_counts())[(df['PERMIT_TYPE'].value_counts()) < 500].index)]
df = df[~df['WORK'].isin((df['WORK'].value_counts())[(df['WORK'].value_counts()) < 1000].index)]
df = df[~df['STRUCTURE_TYPE'].isin((df['STRUCTURE_TYPE'].value_counts())[(df['STRUCTURE_TYPE'].value_counts()) < 500].index)]
df = df[~df['PERMIT_NUM'].isin((df['PERMIT_NUM'].value_counts())[(df['PERMIT_NUM'].value_counts()) > 1].index)]

#print(df.PERMIT_NUM.value_counts(), "\n")
#print(df.POSTAL.value_counts(), "\n")
#print(df.PERMIT_TYPE.value_counts(), "\n")
#print(df.WORK.value_counts(),"\n")
#print(df.STRUCTURE_TYPE.value_counts(), "\n")

pd.set_option('display.max_rows', 25)

perm_num_temp = 2000 + (df['PERMIT_NUM'].astype(str).str[0:2]).astype(int)
perm_num_temp = perm_num_temp - 100*(perm_num_temp > 2050)
app_num_temp = (df['APPLICATION_DATE'].astype(str).str[0:4]).astype(int)
app_year = pd.DataFrame({'APPLICATION_YEAR' : perm_num_temp*(perm_num_temp == app_num_temp)})
#drop under 2010 years

app_datetime = pd.to_datetime(pd.DataFrame({'year' : df['APPLICATION_DATE'].astype(str).str[0:4], 'month' : df['APPLICATION_DATE'].astype(str).str[5:7], 'day' : df['APPLICATION_DATE'].astype(str).str[8:10]}))
issue_datetime = pd.to_datetime(pd.DataFrame({'year' : df['ISSUED_DATE'].astype(str).str[0:4], 'month' : df['ISSUED_DATE'].astype(str).str[5:7], 'day' : df['ISSUED_DATE'].astype(str).str[8:10]}))
app_time_days = pd.to_timedelta((issue_datetime - app_datetime), unit='d')
app_time_days = pd.DataFrame({'TIME_DAYS' : (app_time_days.astype(str).str[0:-5]).astype(int)})
#drop negatives

app_time = app_time_days.astype(str)
app_time = app_time.rename(columns={'TIME_DAYS' : 'PROCESS_TIME'})
app_time['PROCESS_TIME'] = pd.cut(x=app_time_days['TIME_DAYS'], bins=[0, 3, 7, 14, 31, 61, 91, 183, 274, 365, 730, float("inf")],
                                labels=['0-3 DAYS', '4-7 DAYS', '1-2 WEEKS', '3-4 WEEKS', '1-2 MONTHS', '2-3 MONTHS', '3-6 MONTHS', '6-9 MONTHS', '9-12 MONTHS', '1-2 YEARS', 'OVER 2 YEARS'],
                                include_lowest=True)

#print(app_time.value_counts(), "\n")

df = df.drop(['PERMIT_NUM', 'APPLICATION_DATE', 'ISSUED_DATE'], axis=1)
df = pd.concat([app_year, df, app_time], axis=1)
df = df.drop(df[df.APPLICATION_YEAR < 2010].index)
df = df.dropna()
df

,APPLICATION_YEAR,PERMIT_TYPE,STRUCTURE_TYPE,WORK,POSTAL,EST_CONST_COST,PROCESS_TIME
4762,2010,Small Residential Projects,SFD - Detached,Underpinning,M1L,4000,1-2 WEEKS
4763,2010,Small Residential Projects,SFD - Semi-Detached,Other(SR),M4T,1000,1-2 WEEKS
4771,2010,Plumbing(PS),Apartment Building,Backflow Prevention Devices (Water only),M6A,5000,1-2 WEEKS
4773,2010,New Houses,SFD - Detached,New Building,M1E,330000,3-6 MONTHS
4779,2010,Small Residential Projects,SFD - Semi-Detached,Multiple Projects,M5S,35000,0-3 DAYS
...,...,...,...,...,...,...,...
338325,2023,Small Residential Projects,2 Unit - Detached,Second Suite (New),M2M,25000,0-3 DAYS
338331,2023,Small Residential Projects,SFD - Detached,Deck,M4S,3000,3-4 WEEKS
338353,2023,Small Residential Projects,SFD - Detached,Garage,M9R,15000,4-7 DAYS
338405,2023,Building Additions/Alterations,Apartment Building,Interior Alterations,M4Y,10000,1-2 WEEKS
